### Calculate Bandpass Theta for CCF analyses

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 
import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

## import custom functions
import sys
sys.path.append('/home/brooke/pacman/preprocessing/scripts')
import preproc_functions as pf
import roi as subs


In [10]:
# Define theta bandpass filter range
THETA_LOW = 3  # Low cutoff for theta (Hz)
THETA_HIGH = 8  # High cutoff for theta (Hz)

# Define baseline correction period (in seconds relative to trial onset)
BASELINE = (-1.0, 0.0)  # One second before trial onset

# List of subjects
sub_list = ['SLCH002', 'SLCH018', 'BJH016', 'BJH017', 'BJH021', 'BJH025',  'BJH026', 'BJH027', 'BJH029', 'BJH039', 'BJH041', 'BJH046', 'BJH050', 'BJH051', 'LL10', 'LL12', 'LL13', 'LL14', 'LL17', 'LL19']
sub_list = ['LL10', 'LL12', 'LL13', 'LL14', 'LL17', 'LL19']



### By Region Version

In [11]:
# Loop over subjects
for sub in sub_list:
    print(f"Processing subject: {sub}")
    
    # Load the epoched data
    trial_onset_epochs = mne.read_epochs(
        f"/home/brooke/pacman/preprocessing/{sub}/ieeg/{sub}_bp_clean_pres-locked_ieeg.fif"
    )

    if sub == "LL14":
        # get good epochs (for behavioral data only) & Manually add more bad trials because of a bipolar referencing issue that created NANs
        good_epochs = [i for i,x in enumerate(trial_onset_epochs.get_annotations_per_epoch()) if not x and i not in [485, 486, 487, 488, 543, 544, 545, 704, 705, 710, 711, 712, 713, 714]]
        bad_epochs = [i for i,x in enumerate(trial_onset_epochs.get_annotations_per_epoch()) if  x or i in [485, 486, 487, 488, 543, 544, 545, 704, 705, 710, 711, 712, 713, 714]]
    else:
        # get good epochs (for behavioral data only)
        good_epochs = [i for i,x in enumerate(trial_onset_epochs.get_annotations_per_epoch()) if not x]
        bad_epochs = [i for i,x in enumerate(trial_onset_epochs.get_annotations_per_epoch()) if  x]

    # only good epochs
    trial_onset_epochs = trial_onset_epochs[good_epochs]

    # Resample to 1000 
    if trial_onset_epochs.info['sfreq'] > 1000:
        trial_onset_epochs= trial_onset_epochs.resample(1000)
    

    # Dictionary of electrode locations
    ROIs = subs.ROIs[sub]

    # Prep lists
    hc_list, ofc_list, amyg_list, cing_list, mfg_list = [], [], [], [], []

    # Exclude bad ROI from list
    pairs_long_name = [ch.split('-') for ch in trial_onset_epochs.pick_types(seeg=True).info['ch_names']]
    bidx = len(trial_onset_epochs.info['bads']) + 1
    pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

    # Sort ROI into lists
    for ix in range(0, len(pairs_name)):
        if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
            hc_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])
        if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
            ofc_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])
        if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
            amyg_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
            cing_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
            mfg_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])   

    roi_dictionary = {"hc": hc_list, "ofc": ofc_list, "amyg": amyg_list, "cing": cing_list, "mfg": mfg_list}

    for roi_name, electrode_list in roi_dictionary.items():
        if len(electrode_list) == 0:
            continue  # Skip if there are no electrodes for this ROI
        print("Processing ROI:", roi_name)

        # Create a copy of epochs to apply the theta filter
        theta_epochs = trial_onset_epochs.copy().pick_channels(electrode_list)
        
        # Apply bandpass filter for theta (3–8 Hz)
        theta_epochs.filter(
            l_freq=THETA_LOW,
            h_freq=THETA_HIGH,
            picks=electrode_list,
            method="fir",
            fir_window="hamming",
            fir_design="firwin",
            phase="zero",
            filter_length="auto",
            n_jobs = 16
        )

        # Baseline correction
        theta_epochs.apply_baseline(BASELINE)

        # crop and decimate
        if theta_epochs.info['sfreq'] > 600:
            theta_data = theta_epochs.crop(tmin=-1, tmax = 12).decimate(10).get_data() # Shape: (n_epochs, n_channels, n_times)
        else:
            theta_data = theta_epochs.crop(tmin=-1, tmax = 12).decimate(5).get_data()  # Shape: (n_epochs, n_channels, n_times)
        
        # Convert the data to a DataFrame
        times = theta_epochs.times  # Time points
        trial_numbers = good_epochs # Trial indices
        electrodes = theta_epochs.info['ch_names']  # Electrode names

        # Prepare data for saving
        n_trials, n_electrodes, n_times = theta_data.shape
        subject_column = np.repeat(sub, n_trials * n_electrodes * n_times)
        electrode_column = np.tile(np.repeat(electrodes, n_times), n_trials)
        trial_column = np.repeat(trial_numbers, n_electrodes * n_times)
        time_column = np.tile(np.tile(times, len(electrodes)), n_trials)
        value_column = theta_data.reshape(-1)  # Flatten the 3D array into a 1D array

        # Combine into a DataFrame
        df = pd.DataFrame({
            "subject": subject_column,
            "electrode": electrode_column,
            "trial": trial_column,
            "time": time_column,
            "value": value_column
        })

        # Save to CSV
        output_path = f"/home/brooke/pacman//preprocessing/bandpass_theta/{sub}_{roi_name}_baseline_theta.csv"
        df.to_csv(output_path, index=False)
        print(f"Saved CSV for ROI '{roi_name}' to {output_path}")
    


Processing subject: LL10
Reading /home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_clean_pres-locked_ieeg.fif ...


/tmp/ipykernel_250056/1654033260.py:6: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL10/ieeg/LL10_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  trial_onset_epochs = mne.read_epochs(


    Found the data of interest:
        t =   -4000.00 ...   12000.00 ms
        0 CTF compensation matrices available
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Processing ROI: hc
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Setting up band-pass filter from 3 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 3.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 845 samples (1.650 s)



[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 176 tasks      | elapsed:    0.1s


Applying baseline correction (mode: mean)


...

Saved CSV for ROI 'cing' to /home/brooke/pacman//preprocessing/bandpass_theta/LL19_cing_baseline_theta.csv
Processing ROI: mfg
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Setting up band-pass filter from 3 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 3.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 845 samples (1.650 s)



[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 562 tasks      | elapsed:    0.2s


Applying baseline correction (mode: mean)


[Parallel(n_jobs=16)]: Done 688 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 810 out of 810 | elapsed:    0.4s finished
/tmp/ipykernel_250056/1654033260.py:81: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  theta_data = theta_epochs.crop(tmin=-1, tmax = 12).decimate(5).get_data()  # Shape: (n_epochs, n_channels, n_times)


Saved CSV for ROI 'mfg' to /home/brooke/pacman//preprocessing/bandpass_theta/LL19_mfg_baseline_theta.csv
